# Prediction Models

This notebook is the refined prediction model using sklearn pipelines and xgboost cv.

In [1]:
import pandas as pd
import numpy as np
import copy
import pickle
import pyreadstat
import re
import random

from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
import sklearn.metrics as metrics
from sklearn.metrics import auc
from imblearn.pipeline import Pipeline

import xgboost as xgb
from imblearn.over_sampling import SMOTE, RandomOverSampler

# %% cell 1
# load the state abbreviation and population stuff. Pickeld by state_name
with open("pickles\\state_census.pkl", "rb") as f:
    state_census = pickle.load(f)

region = {
    'D1': ['VT', 'RI'],
    'D4': ['ND', 'SD'],
    'D8': ['MT', 'WY'],
    'D9': ['HI', 'AK']
}

# function to handle region to state, and vice versa


def r2s(dict_in):
    dict_out = copy.deepcopy(dict_in)
    for key in dict_in.keys():
        if key in region:
            for st in region[key]:
                dict_out[st] = dict_in[key]
            dict_out.pop(key)
    return dict_out


def s2r(dict_in):
    dict_out = copy.deepcopy(dict_in)
    for key in region:
        dict_out[key] = np.mean([dict_in[st] for st in region[key]])
        for st in region[key]:
            dict_out.pop(st)
    return dict_out

# function that takes region, and gives back state names


def r2slist(st):
    if st in region:
        return region[st]
    else:
        return [st]


def slist2r(st):
    for key in region:
        if st in region[key]:
            return key


def region_explain():
    print(' ')
    print('(D1: VT, RI     D2: ND,SD      D8: MT, WY     D9: HI, AK)')


df_2018, meta_2018 = pyreadstat.read_sas7bdat(
    'NSSRN2018_SAS_encoded_package\\NSSRN_2018_PUF.sas7bdat')
weight = df_2018['RKRNWGTA']

# explore the D's to see if they should be combined
titles = ['Graduated State', 'First Licensed State', 'Current State']
varnames = ['ED_NDLOC_ST_PUF', 'ED_FRN_ST_PUF', 'PN_LOC_ST_PUF']


# combine states into regions
for key in region.keys():
    for st in region[key]:
        for vn in varnames:
            df_2018.loc[df_2018[vn] == st, vn] = key


def explain(name):

    underscore = -name[::-1].find('_')

    if name[underscore:].isnumeric():
        code = name[:underscore-1]
        num = int(name[underscore:])
    else:
        code = name
        num = 0

    try:
        info = survey_key.loc[survey_key['2018 NSSRN VARIABLE NAME'] == code, [
            '2018 NSSRN QUESTIONNAIRE ITEM NUMBER', 'DESCRIPTION']].to_numpy()[0]
        print('{}: {}'.format(info[0], info[1]))

        if num != 0:
            text = survey_key['VALUES'][survey_key['2018 NSSRN VARIABLE NAME'] == code].to_numpy()[
                0]
            ind = [m.start() for m in re.finditer('\n', text)]
            ind.insert(0, -1)
            ind.append(len(text))
            print('    '+text[ind[num-1]+1:ind[num]])
    except:
        print('Does not exist')


survey_key = pd.read_excel('survey_key.xls').replace(np.nan, '')


"""
try a few different approaches:
    only PN_5YRS==1 vs all
    
"""

# df_learn = copy.deepcopy(df_2018).loc[(df_2018['PN_EMPLYD']==1)&(df_2018['PN_NEWEMP']==1)&
#                                       (df_2018['PN_PATCARE']==1)&(df_2018['PN_EHR']!=3),:]
df_learn = copy.deepcopy(df_2018).loc[(df_2018['PN_EMPLYD'] == 1) & (df_2018['PN_5YRS'] == 1) &
                                      (df_2018['PN_PATCARE'] == 1) & (df_2018['PN_EHR'] != 3), :]

leave_reasons = survey_key['2018 NSSRN VARIABLE NAME'][survey_key['2018 NSSRN QUESTIONNAIRE ITEM NUMBER'] == 'C1'].to_numpy()
outside_reasons = ['LE_LVE_DISAB', 'LE_LVE_FAM', 'LE_LVE_LAID',
                   'LE_LVE_COMMTE', 'LE_LVE_RETIRE', 'LE_LVE_SPEMP']
#outside_reasons=['LE_LVE_DISAB', 'LE_LVE_LAID']
inside_reasons = []
for reason in leave_reasons:
    if reason not in outside_reasons:
        inside_reasons.append(reason)

any_outside = np.zeros(df_learn.shape[0])
for reason in outside_reasons:
    any_outside = np.logical_or(
        any_outside, (df_learn[reason] == 1).to_numpy())


df_learn = df_learn.loc[~any_outside, :]


"""droplist"""
# populate dropcols we are gonna use
dropcols = []
#keep = ['B','C','D','E','F','H']
keep = ['B']
remove = ['B2']

# make drop list of unnecessary stuff
for i, row in survey_key.iterrows():

    if row['2018 NSSRN QUESTIONNAIRE ITEM NUMBER'] == '':
        dropcols.append(row['2018 NSSRN VARIABLE NAME'])

    elif ((row['2018 NSSRN QUESTIONNAIRE ITEM NUMBER'] == 'IMP_FLAG') or
          (row['2018 NSSRN QUESTIONNAIRE ITEM NUMBER'] == 'DERIVED') or
          not(row['2018 NSSRN QUESTIONNAIRE ITEM NUMBER'][0] in keep) or
          (row['Action1'] == 'del') or
          (row['2018 NSSRN QUESTIONNAIRE ITEM NUMBER'] in remove)):
        dropcols.append(row['2018 NSSRN VARIABLE NAME'].strip())

# drop stuff in dropcols
df_learn.drop(dropcols, axis=1, inplace=True)


"""bool list"""
# find boolean variables
boolcols = []

for i, row in survey_key.iterrows():
    if row['VALUES'].strip() == '1 ="YES"\n2 ="NO"':
        boolcols.append(row['2018 NSSRN VARIABLE NAME'].strip())

# revalue false from 2 to 0
for col in boolcols:
    if col in df_learn.columns:
        df_learn.loc[df_learn[col] == 2, col] = 0


# %% pre-split processing
y = df_learn['PN_LFTWRK']
X = df_learn.drop(columns=['PN_LFTWRK'])

# remapping
X.loc[X['PN_NEWEMP'] == 1, 'PN_NEWEMP'] = 3/12
X.loc[X['PN_NEWEMP'] == 2, 'PN_NEWEMP'] = 9/12
X.loc[X['PN_NEWEMP'] == 3, 'PN_NEWEMP'] = 1
X.loc[X['PN_NEWEMP'] == 4, 'PN_NEWEMP'] = 0

X.loc[(X["PN_ORIENT"] == 1) & (X["PN_PRECEP"] == 0), "PN_ORIENT"] = 0.5

X.loc[X['PN_EHR'] == 2, 'PN_EHR'] = 0
X.loc[X['PN_EHR'] == 3, 'PN_EHR'] = 0.5

for name in ['PN_WE_TBC', 'PN_WE_IPT', 'PN_WE_HIT', 'PN_OE_CARE', 'PN_OE_DISPL', 'PN_OE_TBC', 'PN_OE_EBC']:
    X.loc[X[name] == 2, name] = 2/3
    X.loc[X[name] == 3, name] = 1/3
    X.loc[X[name] == 4, name] = 0
    X.loc[X[name] == 5, name] = 0

for name in ['PN_THTYP_PTP', 'PN_THTYP_RN', 'PN_THTYP_NP', 'PN_THTYP_OTH_PUF', 'PN_THTYP_PVDTP']:
    X.loc[X[name].isna(), name] = 0

X.loc[X['PN_SATISFD'] == 2, 'PN_SATISFD'] = 2/3
X.loc[X['PN_SATISFD'] == 3, 'PN_SATISFD'] = 1/3
X.loc[X['PN_SATISFD'] == 4, 'PN_SATISFD'] = 0

X['PN_WRK'] += -1
X['PN_MTHPY'] = X['PN_MTHPY']/12
X[['PN_HRS_SCHED_PUF', 'PN_HRS_WRK_PUF']] = X[[
    'PN_HRS_SCHED_PUF', 'PN_HRS_WRK_PUF']]/40
X[['PN_TS_PCC', 'PN_TS_CARE', 'PN_TS_SUPER', 'PN_TS_RESRCH', 'PN_TS_TEACH', 'PN_TS_NNT', 'PN_TS_OTH']] = X[[
    'PN_TS_PCC', 'PN_TS_CARE', 'PN_TS_SUPER', 'PN_TS_RESRCH', 'PN_TS_TEACH', 'PN_TS_NNT', 'PN_TS_OTH']]/100
X[['PN_POP_PNAT', 'PN_POP_NEWB', 'PN_POP_PED', 'PN_POP_ADOL', 'PN_POP_ADLT', 'PN_POP_GER']] = X[[
    'PN_POP_PNAT', 'PN_POP_NEWB', 'PN_POP_PED', 'PN_POP_ADOL', 'PN_POP_ADLT', 'PN_POP_GER']]/100

# addtional drops
# PN_PRECEP: merged to PN_ORIENT
# PN_TELHLTH, PN_THPERS,	PN_THTYP: dont care about telehealth

dropcols2 = ['PN_PRECEP', 'PN_TELHLTH', 'PN_THPERS', 'PN_THTYP_PTP', 'PN_THTYP_RN', 'PN_THTYP_NP',
             'PN_THTYP_OTH_PUF', 'PN_THTYP_PVDTP']  # ,'PN_TS_PCC','PN_POP_PNAT']
X.drop(dropcols2, axis=1, inplace=True)


# check if theres any null
X.isnull().values.any()

# categorical encoders
cat_cols = ['PN_EMPSIT', 'PN_EMPSET', 'PN_LVL_PUF', 'PN_CS_PUF']
cat_drop_vals = []

for name in cat_cols:
    cat_drop_vals.append(name+'_'+str(X[name].value_counts().index[0]))

X = pd.get_dummies(X, columns=cat_cols)

#check correlation
drop_corr =[]
cor = X.corr()
for i, ind in enumerate(X.columns):
    for j, jnd in enumerate(X.columns):
        if i > j and cor.iloc[i, j] > 0.9:
            print('{} and {} have {} correlation'.format(ind, jnd, cor.iloc[i, j]))
            drop_corr.append(random.choice([ind,jnd]))

PN_CS_PUF_6.0 and PN_LVL_PUF_7.0 have 0.9005905917050862 correlation


## Logistic Regression

After encoding features, we find two columns with high correlation. In linear models, one will be randomly chosen to be dropped, in trees, both will be kept.

Two oversampling methods, SMOTE and random oversampling, were used in addtion to the base data. Metric used to evaluate models are AUC

In [2]:
# %% post-split processing and logistic prediction

X_train, X_test, y_train, y_test = train_test_split(
    X.drop(columns=cat_drop_vals+drop_corr), y, test_size=0.2, random_state=2021)

# pipelines
scale_cols = ['PN_MTHPY', 'PN_HRS_SCHED_PUF', 'PN_HRS_WRK_PUF', 'PN_TS_PCC', 'PN_TS_CARE',
              'PN_TS_SUPER', 'PN_TS_RESRCH', 'PN_TS_TEACH', 'PN_TS_NNT', 'PN_TS_OTH', 'PN_POP_PNAT',
              'PN_POP_NEWB', 'PN_POP_PED', 'PN_POP_ADOL', 'PN_POP_ADLT', 'PN_POP_GER', 'PN_EARN_PUF']

lr_preprocess = ColumnTransformer([('scaler', RobustScaler(), scale_cols)
                                   # ,('dummy',OneHotEncoder(drop=cat_drop_vals),cat_cols)
                                   ], remainder='passthrough')


lr_pipeline = Pipeline(steps=[
    ('prep', lr_preprocess),
    ('logistic', LogisticRegression(penalty='l2', max_iter=1000))
])

lr_RO_pipeline = Pipeline(steps=[
    ('prep', lr_preprocess),
    ('OS', RandomOverSampler()),
    ('logistic', LogisticRegression(penalty='l2', max_iter=1000))
])

lr_SMOTE_pipeline = Pipeline(steps=[
    ('prep', lr_preprocess),
    ('OS', SMOTE()),
    ('logistic', LogisticRegression(penalty='l2', max_iter=1000))
])


models = {'none': GridSearchCV(estimator=lr_pipeline, param_grid={'logistic__C': np.logspace(-4, 0.3, 10)}, cv=5, scoring='roc_auc'),
          'RO': GridSearchCV(estimator=lr_RO_pipeline, param_grid={'logistic__C': np.logspace(-4, 0.3, 10)}, cv=5, scoring='roc_auc'),
          'SMOTE': GridSearchCV(estimator=lr_SMOTE_pipeline, param_grid={'logistic__C': np.logspace(-4, 0.3, 10)}, cv=5, scoring='roc_auc'),
          }

results = {}

for key in models:
    models[key].fit(X_train, y_train)
    results[key] = pd.DataFrame(data=models[key].cv_results_)

for key in models:
    print('{}: {}'.format(key, models[key].score(X_test, y_test)))



none: 0.759682226040009
RO: 0.7560028746960237
SMOTE: 0.7557197852738343


No over sampling does better than oversampling.

## XGBoost

No oversampling was used. Pipelines from sklearn were used, but the crossvalidation was done with cv in xgboost.

In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=2021)

xgb_preprocess = ColumnTransformer([('scaler', RobustScaler(), scale_cols)
                                    # ,('onehot',OneHotEncoder(),cat_cols)
                                    ], remainder='passthrough')


xgb_X_train = xgb_preprocess.fit_transform(X_train)
xgb_X_test = xgb_preprocess.transform(X_test)

dtrain = xgb.DMatrix(xgb_X_train, label=y_train)
dtest = xgb.DMatrix(xgb_X_test, label=y_test)

param = {
    'max_depth': 2,
    'learning_rate': 0.1,
    'min_child_weight': 1,
    'gamma': 4,
    'colsample_bytree': 0.9,
    'eval_metric': 'auc',
    'objective': 'binary:logistic'}

""" following snippet was used to try different parameters to tune the best param"""
# for i in range(2, 10):
#     param['min_child_weight'] = i
#     xgb_results = xgb.cv(param, dtrain, num_boost_round=200,
#                          early_stopping_rounds=10, nfold=5)
#     print('{}: {}'.format(i, xgb_results['test-auc-mean'].values[-1]))


xgb_model = xgb.train(param, dtrain, num_boost_round=200)
metrics.roc_auc_score(y_test, xgb_model.predict(dtest))

0.7673447023343116

It performs slightly better than LR